# Course1-Project1: EdTech Backend System

In [79]:
import hashlib
#from enum import Enum
#class Role(Enum):
#    LEARNER = "Learner"
#    INSTRUCTOR = "Instructor"

def set_id(name):
    # Encode the string and compute a hash (SHA-256)
    hash_name = hashlib.sha256(name.encode('utf-8'))
    # Convert the hexadecimal hash to an integer
    id = int(hash_name.hexdigest(), 16)
    # Take only part of it (last 10 digits)
    return id % (10**10)
        
class User:
    counter = 0
    
    def __init__(self, name, role=None, email=None, password=None):
        '''
        'name' and 'role' are required at object creation.
        'role' can be only either 'learner' or 'instructor'
        'email' and 'password' are optional.
        '''
        #if not isinstance(role,Role):
        #    raise ValueError("role must be a member of Role Enum.")
            
        self.name = name
        self.id=set_id(name)
        self.role = role
        self.email = email
 
        if password:
            self.hash_password(password)
            self.dotify_password(password)

        User.counter +=1
        
    def set_email(self, email):
        self.email=email

    def hash_password(self, password):
        '''
        Hash the password using SHA-256 before storing it
        '''
        # encode to bytes and hash with sha256
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        self.__password_hash = hashed
        
    def dotify_password(self, password):
        '''
        Mask the password with dots
        '''
        self.dotified_password = '.' * len(password)

    def validate_password(self,password):
        """
        Check if a given password matches the stored hashed password.
        """
        if self.__password_hash is None:
            return False
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        return hashed == self.__password_hash
        
    def validate_email(self,email):
        return self.email == email            

    def set_role(self):
        option = int(input("Choose either (1) for 'Learner' or (2) for 'Instructor': "))
        if option == 1:
            self.role = "Learner"
        elif option == 2:
            self.role = "Instructor"
        else:
            raise ValueError("Wrong value. Role can only be either (1) for 'Learner' or (2) for 'Instructor")
        return self.role
        
    def print_user_info(self):
        print("Name    :", self.name)
        print("User Id :", self.id)
        print("Role    :", self.role)
        print("Email   :", self.email)
        print("Password:", self.dotified_password)


class Instructor(User):
    def __init__(self, name, taught_courses):
        User.__init__(self, name, "Instructor")
        #self.id = set_id(name) no hace falta porque se genera en el contructor de User
        self.taught_courses = taught_courses # 'taught_courses' is a simple list of all the courses that instructor teaches

    def print_info(self):
        print("Instructor name:",self.name)
        print("Instructor Id  :",self.id)
        print("Taught courses:")
        for course in self.taught_courses:
            print(course)

class Course:
    def __init__(self, name, instructors_list):
        '''Build course from instructors_list'''
        self.name = name
        self.id = set_id(name)
        self.instructors = instructors_list
   
    def print_course_info(self):
        print("Course name:",self.name)
        print("Course Id  :",self.id)
        print("List of instructors:")
        for instructor in self.instructors:
            print(instructor.name, " : ", instructor.id)


class Backend(Course):
    def __init__(self):
        # ToDO: Create instructors_list reading from an excel file
        instructor1 = Instructor(name="Pepe", taught_courses=["A","B","C"])
        instructor2 = Instructor("Felipe",["B","C"])        
        self.instructors_list = [instructor1, instructor2] #'instructors_list' is a simple list of all the instances of Instructor
        
        # Build a dictionary of courses from the list of instructors. key = course_name : value = instructor objects
        self.courses_dict = {} #'courses_dict' is a dictionary to manage instances of 'Course'. course_name -> list of instructors

        for inst in self.instructors_list:
            for course_name in inst.taught_courses:
                if course_name not in self.courses_dict:
                    self.courses_dict[course_name] = [] #crea una entrada con sólo la clave ´course-name' , ojo, en vez de course-id, pero de momento me da igual
                self.courses_dict[course_name].append(inst) # popula la entrada con el objeto del instructor

        # Create a list of Course instances
        self.courses = [] # empty list
        for name, inst in self.courses_dict.items():
            new_course = Course(name,inst)
            self.courses.append(new_course)

    def print_all_instructors(self):
        if not self.instructors_list:
            print("No instructors found.")
            return
        for i in self.instructors_list:
            i.print_info()
            print("-"*30)
    
    def print_all_courses(self):
        '''Display info for all stored courses.'''
        if not self.courses:
            print("No courses found.")
            return
        for i in self.courses:
            i.print_course_info()
            
            print("-"*30)

backend = Backend()
print("List of all available instructors")
backend.print_all_instructors()
print("\nList of all available courses")
backend.print_all_courses()



List of all available instructors
Instructor name: Pepe
Instructor Id  : 7249440833
Taught courses:
A
B
C
------------------------------
Instructor name: Felipe
Instructor Id  : 8437074879
Taught courses:
B
C
------------------------------

List of all available courses
Course name: A
Course Id  : 5871438845
List of instructors:
Pepe  :  7249440833
------------------------------
Course name: B
Course Id  : 7457290844
List of instructors:
Pepe  :  7249440833
Felipe  :  8437074879
------------------------------
Course name: C
Course Id  : 5890963469
List of instructors:
Pepe  :  7249440833
Felipe  :  8437074879
------------------------------
